In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
from tqdm import tqdm

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [ ]:
import sys
sys.path.append('./vggish')

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

In [ ]:
# Project directory
project_name     = 'amsterdam_custom_samples'

In [ ]:
# Set your serval data folder (should be correctly set already)
serval_data_folder = "../data"
project_data_folder = serval_data_folder + '/' + project_name

In [ ]:
# Set input files
input_wav_sample_filepath = project_data_folder + '/csv_files/output_resampled_wav_samples.csv'
target_tfrecord_folder    = project_data_folder + '/tfrecords_all'

In [ ]:
## De vggish model folder
pca_params_vggish_model_folder = serval_data_folder + "/vggish_model/models"
print('Files in vggish model directory: (Expecting vggish_..., youtube_...)')
print(os.listdir(pca_params_vggish_model_folder))

In [ ]:
# Delete flag if it exists
def del_attr(FLAGS, key):
    if key in FLAGS._flags():
        FLAGS.__delattr__(key)

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

del_attr(FLAGS, 'f')
flags.DEFINE_string('f', '', 'kernel')

del_attr(FLAGS, 'audio_embedding_feature_name')
flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

del_attr(FLAGS, 'pca_params')
flags.DEFINE_string(
    'pca_params', pca_params_vggish_model_folder + '/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

del_attr(FLAGS, 'checkpoint')
flags.DEFINE_string(
    'checkpoint', pca_params_vggish_model_folder + '/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

# del_attr(FLAGS, 'yt_checkpoint')
# flags.DEFINE_string(
#    'yt_checkpoint', 'models/youtube_model.ckpt',
#    'Path to the VGGish checkpoint file.')

In [ ]:
# read class labels
df_wav_samples = pd.read_csv(input_wav_sample_filepath, sep=";")
display(df_wav_samples.head())

In [ ]:
## Function that takes examples from wav-file as input and returns a sequence example

## Function is depricated

if False:
    def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
        with tf.Graph().as_default(), tf.Session() as sess:

            # Prepare a postprocessor to munge the model embeddings.
            pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

            # Define the model: load the checkpoint and locate input and output tensors
            # Input: [batch_size, num_frames, num_bands] 
            # where [num_frames, num_bands] represents log-mel-scale spectrogram
            # Output: embeddings
            vggish_slim.define_vggish_slim(training=False)
            vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

            features_tensor = sess.graph.get_tensor_by_name(
                vggish_params.VGGISH_INPUT_TENSOR_NAME)
            embedding_tensor = sess.graph.get_tensor_by_name(
                vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

            # Run inference and postprocessing.
            [embedding_batch] = sess.run([embedding_tensor],
                                         feed_dict={features_tensor: examples_batch})

            print(embedding_batch.shape)
            if(embedding_batch.shape!=10):
                print("*****************************************************************")

            postprocessed_batch = pproc.postprocess(embedding_batch)
            #print(postprocessed_batch)


            ## Maak labels en video-id voor in de example
            label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
            videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

            ## Maak sequence example
            seq_example = tf.train.SequenceExample(
                context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
                feature_lists = tf.train.FeatureLists(
                    feature_list={
                        FLAGS.audio_embedding_feature_name:
                            tf.train.FeatureList(
                                feature=[
                                    tf.train.Feature(
                                        bytes_list=tf.train.BytesList(
                                            value=[embedding.tobytes()]))
                                    for embedding in postprocessed_batch
                                ]
                            )
                    }
                )
            )

        return(seq_example)

In [ ]:
# Code is depricated
if False:
    # Select files
    files = df_wav_samples.filepath.unique()

    for file in tqdm(files):
        ## Fetch labels
        df_labels = df_wav_samples.loc[df_wav_samples.filepath == file].copy(deep=True)

        if len(df_labels) == 0:
            print('[ERROR]: Found sample without labels; filename=', file, sep='')
            continue

        ## Get ind_train
        ind_train = df_labels.ind_train.unique()
        if len(ind_train) != 1 or ind_train[0] not in ['train', 'eval']:
            print('[ERROR]: Found label that is not train or eval; filename=', file, '; ind_train=', df_labels.ind_train, sep='')
            continue

        ## .. encode - Part 1
        examples_batch = vggish_input.wavfile_to_examples(file)

        if(examples_batch.shape[0]>10):
            nr_sec = examples_batch.shape[0]
            print(nr_sec)
            start = int(math.floor((nr_sec-10)/2))
            print("start: " + str(start))
            end = int(nr_sec-math.ceil((nr_sec-10)/2))
            print("end: " + str(end))
            examples_batch = examples_batch[start:end, :, :]
            print(examples_batch.shape)

        ## .. encode - Part 2
        seq_example = getSequenceExample(examples_batch, df_labels.label.unique())

        ## .. write example - Part 3
        if ind_train == 'train':
            tf_record_writer_train.write(seq_example.SerializeToString())
        else:
            tf_record_writer_eval.write(seq_example.SerializeToString())

    tf_record_writer_train.close()
    tf_record_writer_eval.close()

In [ ]:
def convert_and_write_single_wav(file, df_wav_samples, sess, pproc, tf_record_writer_train, tf_record_writer_eval, video_id=[b'-1LrH01Ei1w']):
    ## Fetch labels
    df_labels = df_wav_samples.loc[df_wav_samples.filepath == file].copy(deep=True)
    
    if len(df_labels) == 0:
        print('[ERROR]: Found sample without labels; filename=', file, sep='')
        return

    ## Get ind_train
    ind_train = df_labels.ind_train.unique()
    if len(ind_train) != 1 or ind_train[0] not in ['train', 'eval']:
        print('[ERROR]: Found label that is not train or eval; filename=', file, '; ind_train=', df_labels.ind_train, sep='')
        return

    ## .. encode - Part 1
    examples_batch = vggish_input.wavfile_to_examples(file)

    if(examples_batch.shape[0]>10):
        print(str(file))
        nr_sec = examples_batch.shape[0]
        print(nr_sec)
        start = int(math.floor((nr_sec-10)/2))
        print("start: " + str(start))
        end = int(nr_sec-math.ceil((nr_sec-10)/2))
        print("end: " + str(end))
        examples_batch = examples_batch[start:end, :, :]
        print(examples_batch.shape)
        
    if(examples_batch.shape[0]<10):
        return(False)

    # .. Run inference and postprocessing - Part 2
    features_tensor = sess.graph.get_tensor_by_name(vggish_params.VGGISH_INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

    [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})

    if(embedding_batch.shape!=(10, 128)):
        print(embedding_batch.shape)
        print(str(file))
        print("*****************************************************************")

    # .. Get result - Part 3
    postprocessed_batch = pproc.postprocess(embedding_batch)

    ## .. Maak labels en video-id voor in de example - Part 4
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=df_labels.label.unique()))
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

    ## .. Maak sequence example - Part 5
    seq_example = tf.train.SequenceExample(
        context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
        feature_lists = tf.train.FeatureLists(
            feature_list={
                FLAGS.audio_embedding_feature_name:
                    tf.train.FeatureList(
                        feature=[
                            tf.train.Feature(
                                bytes_list=tf.train.BytesList(
                                    value=[embedding.tobytes()]))
                            for embedding in postprocessed_batch
                        ]
                    )
            }
        )
    )

    ## .. write example - Part 3
    if ind_train == 'train':
        tf_record_writer_train.write(seq_example.SerializeToString())
    else:
        tf_record_writer_eval.write(seq_example.SerializeToString())
        
    return(True)

In [ ]:
## Function that takes examples from wav-file as input and returns a sequence example

def convert_wav_samples(files, df_wav_samples):
    # setup a writer
    ## timestr = time.strftime("%Y%m%d-%H%M%S")
    timestr = time.strftime("%Y%m%d")
    tf_record_writer_train = tf.python_io.TFRecordWriter(target_tfrecord_folder + '/train_' + timestr + '.tfrecord')
    tf_record_writer_eval = tf.python_io.TFRecordWriter(target_tfrecord_folder + '/eval_' + timestr + '.tfrecord')

    corrupt_files = []
    
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        ## .. load and encode file - Part 1
        for file in tqdm(files):
            result = convert_and_write_single_wav(file, df_wav_samples, sess, pproc, tf_record_writer_train, tf_record_writer_eval)
            if not result:
                corrupt_files.append(file)
                
    tf_record_writer_train.close()
    tf_record_writer_eval.close()
    return(corrupt_files)

In [ ]:
files = df_wav_samples.filepath.unique()
skipped_files = convert_wav_samples(files, df_wav_samples)

In [ ]:
display(set(skipped_files))